# KBO
### Selenium을 사용하여 데이터 수집하는 방법을 살펴보겠습니다.


In [2]:
!pip install selenium

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

* 정규식 패턴을 사용합니다. 
    * 아래 정규식 패턴은 playerId의 숫자값을 추출하는 정규식입니다.

In [2]:
pattern = re.compile("playerId=([0-9]+)")

* Selenium을 사용하기 위해서는 브라우저의 드라이버를 설정해야 합니다. 
    * 이번 예제는 크롬 브라우저를 사용하기 때문에 크롬 드라이버를 선택해줍니다. 
    * 다운로드
    * https://chromedriver.chromium.org/downloads

In [3]:
driver = webdriver.Chrome("c:/chrome/chromedriver.exe")

C:\Users\howil\AppData\Local\Temp\ipykernel_3460\1931228805.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("c:/chrome/chromedriver.exe")


In [4]:
driver.get("http://www.koreabaseball.com")

In [ ]:
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(8)
#cphContents_cphContents_cphContents_ddlTeam > option:nth-child(10)

In [7]:
from selenium.webdriver.common.by import By
driver.find_element(By.CSS_SELECTOR,"#lnb > li:nth-child(4) > a").click()

In [8]:
select_team = "#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({})"

In [9]:
select_page = "#cphContents_cphContents_cphContents_ucPager_btnNo{}"

In [10]:
import time

In [11]:
playid = []
for x in range(2,12):
    for_1 = select_team.format(x)
    driver.find_element(By.CSS_SELECTOR, for_1).click()
    time.sleep(2)
    #playid.extend(pattern.findall(driver.page_source))
    for y in range(1,6):
        f2 = select_page.format(y)
        try:
            driver.find_element(By.CSS_SELECTOR, f2).click()
            time.sleep(1)
            playid.extend(pattern.findall(driver.page_source))
        except Exception as e:
            print (e)
        time.sleep(2)

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#cphContents_cphContents_cphContents_ucPager_btnNo5"}
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x002978B3+2193587]
	Ordinal0 [0x00230681+1771137]
	Ordinal0 [0x001441A8+803240]
	Ordinal0 [0x001724A0+992416]
	Ordinal0 [0x0017273B+993083]
	Ordinal0 [0x0019F7C2+1177538]
	Ordinal0 [0x0018D7F4+1103860]
	Ordinal0 [0x0019DAE2+1170146]
	Ordinal0 [0x0018D5C6+1103302]
	Ordinal0 [0x001677E0+948192]
	Ordinal0 [0x001686E6+952038]
	GetHandleVerifier [0x00540CB2+2738370]
	GetHandleVerifier [0x005321B8+2678216]
	GetHandleVerifier [0x003217AA+512954]
	GetHandleVerifier [0x00320856+509030]
	Ordinal0 [0x0023743B+1799227]
	Ordinal0 [0x0023BB68+1817448]
	Ordinal0 [0x0023BC55+1817685]
	Ordinal0 [0x00245230+1856048]
	BaseThreadInitThunk [0x76D7FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77AA7C3E+286]
	RtlGetAppContainerNamedObjectPath [0x77AA7C0E+238]
	(No symbol) [0x00000000]



In [13]:
playid.__len__()

883

In [16]:
import pickle

In [17]:
with open("./kbo.pkl", "wb") as f:
    pickle.dump(playid,f)

### 선수들의 세부 항목들을 수집합니다.

In [28]:
r = requests.get("https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId=62356")

* 이미지를 다운로드 받고 싶다면 아래 코드를 사용합니다.

In [30]:
request.urlretrieve("http:"+BS(r.text).find("div",\
            class_ = "player_basic").find("img")['src'], "./김규민.jpg")

('./김규민.jpg', <http.client.HTTPMessage at 0x16b931fb9a0>)

### 사용자 함수를 작성하여 선수들의 세부 정보를 수집합니다.

In [14]:
def playerID(id_):
    play_dict = {}
    url = "https://www.koreabaseball.com/Record/Player/HitterDetail/Basic.aspx?playerId={}".format(id_)
    r = requests.get(url)
    #print (r.url)
    for x in BS(r.text).find("div",\
                class_ = "player_basic").findAll("li"):
        rt = [y.strip() for y in x.text.strip().split(":")]
        play_dict[rt[0]] = rt[1]
    play_dict['Team'] = BS(r.text).find("h4", class_="team regular/2021/emblem_NC").text
    return play_dict

In [15]:
a = pd.DataFrame([playerID(playid[2])])

In [16]:
a

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,Team
0,강진성,No.49,1993년 10월 19일,내야수(우투우타),180cm/81kg,가동초-잠신중-경기고-NC-경찰,7000만원,12000만원,12 NC 4라운드 33순위,12NC,NC 다이노스


In [48]:
b = pd.DataFrame([playerID(playid[3])])

In [49]:
pd.concat([a,b])

,선수명,등번호,생년월일,포지션,신장/체중,경력,입단 계약금,연봉,지명순위,입단년도,Team
0,강진성,No.49,1993년 10월 19일,내야수(우투우타),180cm/81kg,가동초-잠신중-경기고-NC-경찰,7000만원,12000만원,12 NC 4라운드 33순위,12NC,NC 다이노스
0,강태경,No.78,2001년 07월 26일,투수(우투좌타),188cm/95kg,양덕초-잠실중-배명고,7000만원,3000만원,20 NC 2차 5라운드 41순위,,NC 다이노스
